In [44]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY=os.getenv("GOOGLE_API_KEY")

YOUTUBE_VIDEO="https://www.youtube.com/watch?v=cpLi-82koFM"

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI

model=ChatGoogleGenerativeAI(model="gemini-pro")

In [15]:
result=model.invoke("What is capital of india")
print(result.content)

New Delhi


In [22]:
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()

chain=model | parser
chain.invoke("What is capital of india")

'New Delhi'

In [23]:
from langchain.prompts import ChatPromptTemplate

template="""
Answer the question based on the context below. If you can't answer the question, reply "I don't know".

Context:{context}
 
Question:{question}
"""
prompt=ChatPromptTemplate.from_template(template)
prompt.format(context="Mary's sister is Susana",question="Who is Mary's sister?")

'Human: \nAnswer the question based on the context below. If you can\'t answer the question, reply "I don\'t know".\n\nContext:Mary\'s sister is Susana\n \nQuestion:Who is Mary\'s sister?\n'

In [24]:
chain= prompt | model | parser
chain.invoke({
            "context":"Mary's sister is Susana",
            "question":"Who is Mary's sister?"
})

'Susana'

### Combining Chains

In [28]:
translation_prompt=ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

In [35]:
from operator import itemgetter

translation_chain=(
    {"answer":chain,"language":itemgetter("language")} | translation_prompt | model | parser 
)
translation_chain.invoke(
    {
        "context":"Mary's sister is Susana. She doesn't have any more siblings.",
        "question":"Who is Mary's sisters? does Mary have one?",
        "language":"Spanish",
    }
)

'La hermana de Mary es Susana. Sí, Mary tiene una hermana.'

In [61]:
import tempfile
import whisper
from pytube import YouTube
# Let's do this only if we haven't created the transcription file yet.

if not os.path.exists("transcription.txt"):

    audio_file = YouTube(YOUTUBE_VIDEO).streams.filter(only_audio=True).first().download(filename="audio.mp4")

    whisper_model = whisper.load_model("tiny")
    transcription = whisper_model.transcribe(audio_file)
    result = whisper_model.transcribe("audio_english.mp3")
    print(result["text"])

RegexMatchError: get_throttling_function_name: could not find match for multiple